# 2022 NDVI for Marina, Mission, North Beach in San Francisco

2022 NDVI statistics were calculated for three popular neighborhoods in San Francisco to assess the greenness and overall suitability for living in each neighborhood. Presidio was added as a benchmark. Nothing conclusive, but it's fun to just look from this perspective!

In [13]:
import os
import shutil
from glob import glob


import earthpy as et
import earthpy.earthexplorer as etee
import earthpy.spatial as es
import geopandas as gpd
import geoviews as gv
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from shapely.geometry import Point
from bokeh.models import HoverTool
import warnings


hv.extension('bokeh')
warnings.filterwarnings('ignore')


data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
sfo_dir = os.path.join(data_dir, 'sfo-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'sfo-green-space', 'processed')

for dir in [data_dir, sfo_dir, ndvi_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir)

In [14]:
sfo_path = os.path.join(sfo_dir, 'sfo-neighborhoods.shp')
if not os.path.exists(sfo_path):
    sfo_url = (
        "https://data.sfgov.org"\
        "/api/geospatial/pty2-tcw4?method=export&format=Shapefile"
    )
    gpd.read_file(sfo_url).to_file(sfo_path)

sfo_gdf = gpd.read_file(sfo_path).set_index('nhood')
neigh_gdf = (
    sfo_gdf
    .loc[['Marina', 'Mission', 'North Beach', 'Presidio']]
)
# neigh_gdf

In [15]:
locations_gdf = neigh_gdf.copy()
locations_gdf['centroid'] = locations_gdf['geometry'].centroid
locations_gdf['longitude'] = locations_gdf['centroid'].x
locations_gdf['latitude'] = locations_gdf['centroid'].y
locations_gdf = locations_gdf.drop(columns=['centroid', 'geometry'])
locations_gdf = locations_gdf.reset_index()
# locations_gdf

In [16]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date

    Download data from the National Agriculture Imagery Program (NAIP)
    given a spatial and temporal extent. 

    <citation>

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    # print(f"Neighborhood: {name}")
    label = name.lower().replace(' ', '-')
    bbox = etee.BBox(*geometry.bounds)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset='NAIP',
        label=label,
        bbox=bbox,
        start=start,
        end=end,
        store_credential=True
    )
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)

    return naip_downloader

In [17]:
def load_and_merge_arrays(neighborhood):
    """
    Load in and merge downloaded arrays

    Parameters
    ==========
    neighborhood : str
      The name used to label the download
    
    Returns
    =======
    merge_da : rxr.DataArray
      DataArray with the merged data
    
    """

    # print(f'Neighborhood: {neighborhood}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        neighborhood.lower().replace(' ', '-')
    )
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked=True) for tp in tif_paths]
    merged_da = merge_arrays(das)
    return merged_da

In [18]:
def calculate_ndvi_statistics(gdf, da, stats_path, override=False):
    """
    Calculate NDVI statistics, then summarize and save statistics

    Uses downloaded National Agriculture Imagery Program (NAIP) data.
    <citation>

    Parameters
    ==========
    gdf : gdf.GeoDataFrame
      One row with the neighborhood name and boundary
    da : rxr.DataArray
      Multispectral (NAIP) raster data
    stats_path : path-like
      The path to save the statistics to
    """
    name = str(gdf.index[0])
    # print(f'Neighborhood: {name}')

    file_is_empty = True
    if os.path.exists(stats_path):
        # print('Stats file exists.')
        stats_df = pd.read_csv(stats_path)
        file_is_empty = len(stats_df) == 0
        # print(f'Stats file is empty? {file_is_empty}')

        if not file_is_empty:
            print(name)
            print(list(stats_df.neighborhood))
            print(name in list(stats_df.neighborhood))
            if (name in list(stats_df.neighborhood)) and (not override):
                # print('Neighborhood stats have already been calculated. Skipping...')
                return

    reprojected_gdf = gdf.to_crs(da.rio.crs)

    naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)
    naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)

    ndvi_da = (
        (da.sel(band=4) - da.sel(band=1))
        / (da.sel(band=4) + da.sel(band=1))
    )

    # print('Writing NDVI statistics to file...')
    pd.DataFrame(dict(
        neighborhood=[name],
        ndvi_min=[float(ndvi_da.min())],
        ndvi_max=[float(ndvi_da.max())],
        ndvi_median=[float(ndvi_da.median())],
        ndvi_25pctl=[np.nanpercentile(ndvi_da, 25)],
        ndvi_75pctl=[np.nanpercentile(ndvi_da, 75)],
        ndvi_mean=[float(ndvi_da.mean())],
        ndvi_std=[float(ndvi_da.std())]
    )).to_csv(stats_path, mode='a', header=file_is_empty, index=False)

In [19]:
ndvi_stats_path = os.path.join(ndvi_dir, 'sfo-neighborhood-ndvi-stats.csv')


for neighborhood_name, details in neigh_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        # print('NDVI statistics file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
    
    if neighborhood_name in ndvi_stats_df.index:
        # print('Neighborhood stats have already been calculated. Skipping...')
        continue

    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2022-01-01', '2022-12-31')
    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(
        sfo_gdf.loc[[neighborhood_name]],
        merged_da,
        ndvi_stats_path
    )

    shutil.rmtree(downloader.data_dir)

ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
ndvi_stats_df

,ndvi_min,ndvi_max,ndvi_median,ndvi_25pctl,ndvi_75pctl,ndvi_mean,ndvi_std
neighborhood,,,,,,,
Mission,-0.979798,0.979592,-0.074434,-0.151515,-0.028369,-0.077182,0.152398
Marina,-0.979798,0.979592,-0.157143,-0.293233,-0.039514,-0.133525,0.215500
North Beach,-0.979798,0.979592,-0.250000,-0.324138,-0.072100,-0.195221,0.166279
Presidio,-0.967742,0.977654,-0.075862,-0.245614,0.051852,-0.070517,0.239048


In [20]:
title = "San Francisco NDVI Chloropleth"
legend = "NDVI 75-Percentile"
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
chloropleth = gv.tile_sources.EsriImagery * gv.Polygons(
    sfo_gdf.join(ndvi_stats_df, how='left'),
    vdims=['ndvi_75pctl'] 
).opts(
    title=title,
    colorbar=True,
    tools=['hover'],
    cmap='viridis_r',
    colorbar_opts={'title': legend},
    width=1200,
    xaxis="bare", yaxis="bare"
)

# Create the location indicator plot
location_markers = gv.Points(
    locations_gdf, vdims=['nhood']
).opts(
    color='red',  # You can customize the marker color
    size=5,       # You can customize the marker size
    tools=['hover'],
)

# Combine both plots using overlay
combined_plot = chloropleth * location_markers

# Define hover tool for location markers
hover = HoverTool()
hover.tooltips = [("Neighborhood", "@neighborhood")]

# Apply the hover tool to the location marker plot
combined_plot = combined_plot.opts(tools=[hover])

# Show the combined plot
# combined_plot

In [21]:
combined_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (ndvi_75pctl)
   .Points.I   :Points   [longitude,latitude]   (nhood)

In [22]:
hv.save(combined_plot, 'sfo-ndvi-chloropleth.html')

**Data Citation**

Mayor’s Office of Neighborhood Services. (2019). SF Find Neighborhoods data. Retrieved from  https://data.sfgov.org/Geographic-Locations-and-Boundaries/SF-Find-Neighborhoods/pty2-tcw4 

U.S. Department of Agriculture, Farm Service Agency. (2022). National Agriculture Imagery Program (NAIP) data. Retrieved from https://naip-usdaonline.hub.arcgis.com/

In [23]:
%%capture
%%bash
jupyter nbconvert san-francisco-multispectral.ipynb --to html --no-input